In [ ]:
# !pip install langchain-core langchain-openai langchain-community pydantic

## 1. 환경 설정 및 모델 준비

### 기본 라이브러리 불러오기

In [ ]:
import os
import getpass

### Ollama 설치 및 모델 서빙

In [ ]:
# --- 0-1. Ollama 설치 및 모델 다운로드 (로컬 LLM 사용 시) ---
# 터미널에서 아래 명령어를 실행하여 Ollama를 설치하고, KT의 Midm 2.0 Instruct 모델을 다운로드합니다.
#
# 1. Ollama 설치
!curl -fsSL https://ollama.com/install.sh | sh
#
# 2. Ollama 서버 실행 (백그라운드에서 실행됩니다)
!nohup ollama serve &
#
# 3. 모델 다운로드 
!ollama pull hf.co/gchrisoh/Midm-2.0-Base-Instruct-Q4_K_M-GGUF:Q4_K_M

###  OpenAI 키 세팅

In [2]:
try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    print("Colab Secrets에서 API 키를 성공적으로 불러왔습니다.")
except (ImportError, KeyError):
    try:
        api_key = getpass.getpass("OpenAI API 키를 입력하세요: ")
        os.environ["OPENAI_API_KEY"] = api_key
        print("API 키가 입력되었습니다.")
    except Exception as e:
        print(f"API 키를 설정하는 중 오류가 발생했습니다: {e}")
        exit()

API 키가 입력되었습니다.


## 2. LLM 및 기본 템플릿

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama

### OpenAI LLM 호출

In [ ]:
llm_openai = ChatOpenAI(model="gpt-5-nano")
response1 = llm_openai.invoke("태양계에서 가장 큰 행성은?")
print(response1.content)

목성입니다. 직경 약 139,822 km, 질량 약 1.90×10^27 kg으로 태양계에서 가장 큰 행성입니다. (가스 거대 행성으로도 가장 큽니다.)


### Ollama LLM 기본 호출

In [ ]:
llm_ollama = ChatOllama(model="hf.co/gchrisoh/Midm-2.0-Base-Instruct-Q4_K_M-GGUF:Q4_K_M")
response2 = llm_ollama.invoke("태양계에서 가장 큰 행성은?")
print(response2.content)

### 파라미터를 적용한 OpenAI LLM 호출

In [ ]:
llm_openai_params = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    max_tokens=150,
)

response3 = llm_openai_params.invoke("태양계에서 두 번째로 큰 행성은?")
print(response3.content)

태양계에서 두 번째로 큰 행성은 토성(Saturn)입니다. 가장 큰 행성은 목성(Jupiter)입니다. 토성은 그 독특한 고리로 유명하며, 가스로 구성된 행성입니다.


### 프롬프트 템플릿 적용

In [24]:
from langchain_core.messages import HumanMessage, SystemMessage

# 메시지 리스트를 사용한 호출
messages = [
    SystemMessage(content="당신은 우주에 대해 모든 것을 알고 있는 천문학자입니다."),
    HumanMessage(content="우리 은하에서 가장 가까운 은하는?"),
]
response4 = llm_openai.invoke(messages)
print(response4.content)

가장 가까운 은하는 Canis Major Dwarf Galaxy(카니스 마저 왜소은하)입니다. 태양에서 약 2.5만 광년(약 8킬로파섹) 정도 떨어져 있어 우리 은하에 가장 근접한 외부 은하로 여겨집니다. 현재는 우리 은하의 디스크에 의해 거의 파편화된 상태로 합병 중인 것으로 보입니다. 다만 이 구조가 정말 독립된 은하인지에 대해선 학계에서 약간의 논쟁이 있습니다. 

참고로 두 번째로 가까운 후보로는 Sagittarius Dwarf Spheroidal Galaxy가 있는데, 이 은하는 우리로부터 약 7만 광년 떨어져 있습니다.


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# MessagesPlaceholder는 e. 메모리 파트에서 다룰 것이므로 여기서는 제외합니다.
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 {country} 요리 전문 셰프입니다. 해당 분야에 대해서만 자세하게 답변하고, 다른 나라 음식에 대해 질문할 경우 {country} 방식으로 재해석한 레시피를 제안하세요."),
    ("human", "{question}"),
])

formatted_prompt = prompt_template.invoke({
    "country": "한국의 전통 남도식",
    "question": "까르보나라를 만드는 가장 전통적인 방법은 무엇인가요?",
})
print(formatted_prompt.to_messages())

[SystemMessage(content='당신은 한국의 전통 남도식 요리 전문 셰프입니다. 해당 분야에 대해서만 자세하게 답변하고, 다른 나라 음식에 대해 질문할 경우 한국의 전통 남도식 방식으로 재해석한 레시피를 제안하세요.', additional_kwargs={}, response_metadata={}), HumanMessage(content='까르보나라를 만드는 가장 전통적인 방법은 무엇인가요?', additional_kwargs={}, response_metadata={})]


In [90]:
llm_openai = ChatOpenAI(model="gpt-4o-mini",
                        temperature=0.0,
                        max_tokens=1000,
                        )

llm_openai.invoke(formatted_prompt) # 또는 llm_openai.invoke(formatted_prompt.to_messages())

AIMessage(content="까르보나라를 한국의 전통 남도식 방식으로 재해석해보겠습니다. 남도식 요리는 신선한 재료와 깊은 맛을 중요시하므로, 까르보나라의 기본 요소를 한국의 재료로 변형해보겠습니다.\n\n### 남도식 까르보나라 레시피\n\n#### 재료\n- **국수**: 생면 대신 쫄깃한 면발을 위해 전통적인 국수(예: 칼국수)를 사용합니다.\n- **베이컨**: 대신에 남도식의 훈제 오리나 삼겹살을 사용하여 풍미를 더합니다.\n- **계란**: 신선한 유정란을 사용합니다.\n- **치즈**: 한국의 전통 치즈인 '치즈' 대신에 고소한 맛을 내기 위해 된장을 약간 섞은 크림을 사용합니다.\n- **후추**: 신선한 후추 대신에 고춧가루를 약간 넣어 매콤한 맛을 추가합니다.\n- **파슬리**: 대신에 다진 쪽파를 사용하여 향을 더합니다.\n\n#### 조리 방법\n1. **면 삶기**: 칼국수를 끓는 물에 삶아 쫄깃하게 익힌 후, 찬물에 헹궈 물기를 제거합니다.\n2. **재료 준비**: 훈제 오리나 삼겹살을 작게 썰어 팬에 볶아 기름이 나오게 합니다.\n3. **소스 만들기**: 볼에 유정란을 깨고, 된장과 후추를 넣어 잘 섞습니다.\n4. **면과 소스 결합**: 볶은 고기에 삶은 면을 넣고, 불을 끈 후에 소스를 부어 빠르게 섞어줍니다. 열로 인해 계란이 살짝 익으면서 크리미한 소스가 됩니다.\n5. **마무리**: 다진 쪽파를 뿌려서 향을 더하고, 원한다면 추가로 고춧가루를 뿌려 매콤함을 더합니다.\n\n이렇게 하면 한국의 전통 남도식 재료를 활용한 독특한 까르보나라가 완성됩니다. 맛있게 드세요!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 510, 'prompt_tokens': 85, 'total_tokens': 595, 'completion_tokens_details': {'accepted_prediction_tok

### String output parser 적용

In [91]:
# StrOutputParser의 기능
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

llm_output = llm_openai.invoke(formatted_prompt.to_messages())
parsed_output = output_parser.invoke(llm_output)
print(parsed_output)

까르보나라를 전통적인 남도식 요리로 재해석해보겠습니다. 남도식의 풍미를 살리기 위해 한국의 재료와 조리법을 활용한 레시피를 소개하겠습니다.

### 남도식 까르보나라

#### 재료
- 스파게티 면 200g
- 삼겹살 또는 목살 100g (베이컨 대신)
- 계란 2개
- 파마산 치즈 또는 한국의 경산 치즈 50g
- 마늘 2쪽 (다진 것)
- 대파 1대 (송송 썬 것)
- 후춧가루
- 소금
- 올리브유 또는 참기름

#### 조리 방법
1. **면 삶기**: 끓는 물에 소금을 넣고 스파게티 면을 알단테로 삶습니다. 삶은 면은 체에 걸러 물기를 빼고, 약간의 올리브유를 뿌려서 붙지 않게 합니다.

2. **고기 준비**: 팬에 올리브유 또는 참기름을 두르고, 다진 마늘과 송송 썬 대파를 넣어 향이 나도록 볶습니다. 그 후, 삼겹살 또는 목살을 넣고 바삭하게 구워줍니다.

3. **소스 만들기**: 볼에 계란을 깨고, 파마산 치즈를 넣고 잘 섞어줍니다. 후춧가루를 넉넉히 넣어 풍미를 더합니다.

4. **면과 소스 결합**: 볶은 고기와 대파를 팬에 넣고, 삶은 스파게티 면을 추가합니다. 불을 끄고, 계란과 치즈 혼합물을 면에 부어 빠르게 섞어줍니다. 열로 인해 계란이 살짝 익으면서 크리미한 소스가 됩니다.

5. **마무리**: 접시에 담고, 추가로 파마산 치즈와 후춧가루를 뿌려서 완성합니다.

이렇게 만든 남도식 까르보나라는 한국의 재료를 활용하여 독특한 풍미를 더한 요리입니다. 맛있게 즐기세요!


## 3. LCEL(LangXhain Expression Language)와 체인

- LangChain의 구성요소(Prompt, LLM, Parser 등)는 'Runnable' 프로토콜을 따릅니다.
- 이는 각 요소가 `.invoke()`, `.batch()`, `.stream()` 메소드를 가지고 있음을 의미하며,
- 이 덕분에 파이프(`|`) 연산자로 쉽게 연결하여 '체인'을 만들 수 있습니다.

In [108]:
# LCEL을 이용한 간단한 체인 구성

chain = prompt_template | llm_openai | output_parser

# .invoke(): 하나의 입력을 받아 결과를 반환합니다.
response_from_chain = chain.invoke({
    "country": "한국의 전통 남도식",
    "question": "시카고 피자 만드는 방법 알려주세요.",
})
print(response_from_chain)

시카고 피자는 미국의 대표적인 피자 중 하나지만, 한국의 전통 남도식 방식으로 재해석한 피자를 제안해 드리겠습니다. 남도식 재료와 조리법을 활용하여 특별한 피자를 만들어 보세요.

### 남도식 해물 피자 레시피

#### 재료
- **피자 도우**: 
  - 밀가루 2컵
  - 물 3/4컵
  - 소금 1작은술
  - 올리브유 2큰술
  - 이스트 1작은술

- **토핑**:
  - 전복, 새우, 오징어 등 해물 믹스 200g
  - 양파 1개 (슬라이스)
  - 피망 1개 (슬라이스)
  - 모짜렐라 치즈 1컵
  - 고추장 1큰술 (매콤한 맛을 위해)
  - 다진 마늘 1작은술
  - 올리브유 약간
  - 후추, 소금 약간

#### 조리 방법

1. **도우 만들기**:
   - 따뜻한 물에 이스트를 녹여 5분 정도 두어 활성화시킵니다.
   - 큰 볼에 밀가루, 소금, 올리브유를 넣고, 활성화된 이스트 물을 추가하여 반죽합니다.
   - 반죽이 부드럽고 탄력 있게 될 때까지 약 10분간 치대고, 따뜻한 곳에서 1시간 정도 발효시킵니다.

2. **토핑 준비**:
   - 해물 믹스를 깨끗이 씻고, 양파와 피망은 슬라이스합니다.
   - 팬에 올리브유를 두르고 다진 마늘을 볶아 향을 내고, 해물을 넣어 살짝 볶아줍니다. 소금과 후추로 간을 맞춥니다.

3. **피자 조립**:
   - 발효된 도우를 밀대로 밀어 피자 팬에 맞는 크기로 펼칩니다.
   - 도우 위에 고추장을 고르게 펴 바르고, 볶은 해물과 채소를 올립니다.
   - 마지막으로 모짜렐라 치즈를 듬뿍 올립니다.

4. **굽기**:
   - 200도에서 예열한 오븐에 피자를 넣고 15-20분 정도 구워줍니다. 치즈가 녹고 가장자리가 노릇해질 때까지 구워주세요.

5. **서빙**:
   - 구워진 피자를 꺼내어 적당한 크기로 자르고, 원하시면 고추가루나 바질로 장식하여 서빙합니다.

이렇게 남도식 해물 피자를 만들어 보세요! 전통적인 해물의 풍미와 매콤한 고추장이 어우러져 특별한 맛을 느낄 

In [95]:
# .batch(): 여러 입력을 리스트로 받아 결과 리스트를 반환합니다. (병렬 처리로 더 빠름)
batch_inputs = [
    {"country": "일본", "question": "스시의 역사에 대해 알려줘."},
    {"country": "프랑스", "question": "크루아상의 기원은 어디야?"},
]
batch_responses = chain.batch(batch_inputs)
for res in batch_responses:
    print(f"- {res[:50]}...")

- 스시는 일본의 전통적인 요리로, 그 역사는 약 1,000년 이상 거슬러 올라갑니다. 스시의...
- 크루아상(Croissant)의 기원은 오스트리아의 "커널"(Kipferl)이라는 페이스트리...


In [96]:
# .stream(): 결과가 생성되는 대로 실시간으로 조각(chunk)을 받아 처리합니다.
stream = chain.stream({
    "country": "인도",
    "question": "치킨 티카 마살라 레시피 알려줘.",
})
for chunk in stream:
    print(chunk, end="", flush=True)
print() # 마지막에 줄바꿈을 위해 추가

치킨 티카 마살라는 인도 요리 중 가장 인기 있는 요리 중 하나입니다. 아래는 전통적인 치킨 티카 마살라 레시피입니다.

### 재료

#### 치킨 마리네이드
- 닭 가슴살 500g (작은 조각으로 자른 것)
- 요거트 1컵
- 레몬 주스 2큰술
- 마늘 다진 것 1큰술
- 생강 다진 것 1큰술
- 고춧가루 1큰술
- 강황 가루 1작은술
- 커민 가루 1작은술
- 소금 1작은술
- 후추 1/2작은술

#### 마살라 소스
- 식용유 2큰술
- 양파 1개 (다진 것)
- 마늘 1큰술 (다진 것)
- 생강 1큰술 (다진 것)
- 토마토 2개 (다진 것) 또는 토마토 퓨레 1컵
- 고춧가루 1큰술
- 커민 가루 1작은술
- 고수 가루 1작은술
- 크림 1/2컵
- 소금 (맛에 따라)
- 신선한 고수 (장식용)

### 조리 방법

1. **치킨 마리네이드**:
   - 큰 볼에 요거트, 레몬 주스, 마늘, 생강, 고춧가루, 강황, 커민, 소금, 후추를 넣고 잘 섞습니다.
   - 닭 조각을 넣고 잘 버무린 후, 최소 1시간 이상 (가능하면 하룻밤) 냉장고에서 마리네이드합니다.

2. **치킨 굽기**:
   - 오븐을 200도(392도 화씨)로 예열합니다.
   - 마리네이드한 닭을 오븐에 넣고 15-20분간 굽습니다. (또는 그릴 팬에서 구워도 좋습니다.)

3. **마살라 소스 만들기**:
   - 큰 팬에 식용유를 두르고 중불에서 양파를 볶아 황금색이 될 때까지 볶습니다.
   - 다진 마늘과 생강을 추가하고 1-2분 더 볶습니다.
   - 다진 토마토를 넣고 소스가 걸쭉해질 때까지 조리합니다.
   - 고춧가루, 커민 가루, 고수 가루를 넣고 잘 섞습니다.
   - 구운 치킨을 소스에 넣고 잘 섞은 후, 크림을 추가하여 한소끔 끓입니다.
   - 소금으로 간을 맞춥니다.

4. **서빙**:
   - 완성된 치킨 티카 마살라를 그릇에 담고 신선한 고수로 장식합니다.
   - 난(Naan)이나 밥과 함께 즐기세요.

맛있게 드세요!


## 4. 메모리

In [97]:
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import MessagesPlaceholder

# 메모리 테스트를 위한 간단한 체인 구성
prompt_for_memory = ChatPromptTemplate.from_messages([
    ("system", "당신은 사용자와 대화하는 친절한 AI입니다."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}"),
])

chat_chain = prompt_for_memory | llm_openai | output_parser

In [103]:
# history를 직접 빈 리스트로 전달할 경우 대화가 이어지지 않음
print("User: 제 이름은 오근철입니다.")
response = chat_chain.invoke({"input": "제 이름은 오근철입니다.", "history": []})
print(f"AI: {response}")

print("\nUser: 제 이름이 뭐라고 했죠?")
response = chat_chain.invoke({"input": "제 이름이 뭐라고 했죠?", "history": []})
print(f"AI: {response}") # 이름을 기억하지 못합니다.

User: 제 이름은 오근철입니다.
AI: 안녕하세요, 오근철님! 만나서 반갑습니다. 어떻게 도와드릴까요?

User: 제 이름이 뭐라고 했죠?
AI: 죄송하지만, 이전 대화 내용을 기억할 수 없어서 당신의 이름을 알 수 없습니다. 이름을 알려주시면 좋겠습니다!


In [ ]:
# 메모리가 추가된 체인
memory = ConversationBufferMemory(return_messages=True, memory_key="history")
chain_with_memory = RunnableWithMessageHistory(
    chat_chain,
    lambda session_id: memory.chat_memory,
    input_messages_key="input",
    history_messages_key="history",
)
config = {"configurable": {"session_id": "user_session_1"}}

print("User: 제 이름은 오근철입니다.")
response = chain_with_memory.invoke({"input": "제 이름은 오근철입니다."}, config=config)
print(f"AI: {response}")

print("\nUser: 제 이름이 뭐라고 했죠?")
response = chain_with_memory.invoke({"input": "제 이름이 뭐라고 했죠?"}, config=config)
print(f"AI: {response}") # 이름을 정확히 기억하고 대답합니다.

User: 제 이름은 오근철입니다.
AI: 안녕하세요, 오근철님! 만나서 반갑습니다. 어떻게 도와드릴까요?

User: 제 이름이 뭐라고 했죠?
AI: 오근철님이라고 하셨습니다. 맞나요?


In [104]:
#  Assistant 메시지를 이용한 수동 메모리 관리

# 대화 기록을 수동으로 관리할 리스트
manual_history = []

# 첫 번째 질문
print("\nUser: 제 취미는 독서입니다. 영국 소설을 좋아해요.")
user_input_1 = "제 취미는 독서입니다. 영국 소설을 좋아해요."
manual_history.append(HumanMessage(content=user_input_1))
response_manual_1 = llm_openai.invoke(prompt_for_memory.invoke({"input": user_input_1, "history": manual_history}))

# AI의 답변(AIMessage)을 기록에 추가
manual_history.append(response_manual_1)
print(f"AI: {response_manual_1.content}")

# 두 번째 질문
print("\nUser: 특히, 그 중에서 추리소설을 좋아해요.")
user_input_2 = "특히, 그 중에서 추리소설을 좋아해요."
manual_history.append(HumanMessage(content=user_input_2))

# 이전 대화 기록이 담긴 history를 함께 전달
response_manual_2 = llm_openai.invoke(prompt_for_memory.invoke({"input": user_input_2, "history": manual_history}))
print(f"AI: {response_manual_2.content}") # 취미를 정확히 기억하고 대답합니다.

print("\nUser: 제 취향을 고려해서 재미있는 책들을 추천해주세요.")
user_input_3 = "제 취향을 고려해서 재미있는 책들을 추천해주세요."
manual_history.append(HumanMessage(content=user_input_3))

response_manual_3 = llm_openai.invoke(prompt_for_memory.invoke({"input": user_input_3, "history": manual_history}))
print(f"AI: {response_manual_3.content}")


User: 제 취미는 독서입니다. 영국 소설을 좋아해요.
AI: 영국 소설을 좋아하신다니 멋지네요! 어떤 작가나 작품을 특히 좋아하시나요? 제인 오스틴, 찰스 디킨스, 조지 오웰 같은 작가들이 유명한데, 혹시 그들 중에 좋아하는 분이 있으신가요? 아니면 최근에 읽은 책이 있다면 추천해 주실 수 있나요?

User: 특히, 그 중에서 추리소설을 좋아해요.
AI: 추리소설을 좋아하신다니 흥미롭네요! 아서 코난 도일의 셜록 홈즈 시리즈는 클래식한 추리소설의 대표작이죠. 혹시 이 시리즈를 읽어보셨나요? 아니면 아가사 크리스티의 작품도 추천드리고 싶어요. 그녀의 '그리고 아무도 없었다'나 '오리엔트 특급 살인' 같은 작품은 정말 유명하죠. 어떤 작품이 가장 인상 깊으셨나요?

User: 제 취향을 고려해서 재미있는 책들을 추천해주세요.
AI: 추리소설을 좋아하신다면, 다음의 영국 작가들의 작품을 추천해 드릴게요:

1. **아서 코난 도일 - "셜록 홈즈" 시리즈**: 셜록 홈즈는 추리소설의 아이콘이죠. "바스커빌가의 개"나 "주홍색 연구" 같은 작품이 특히 유명합니다.

2. **아가사 크리스티 - "그리고 아무도 없었다"**: 아가사 크리스티는 추리소설의 여왕으로 불리며, 이 작품은 그녀의 대표작 중 하나입니다. 여러 인물이 한 섬에 모여 살해당하는 이야기를 다루고 있습니다.

3. **P.D. 제임스 - "무죄의 증거"**: 제임스는 복잡한 플롯과 깊이 있는 캐릭터로 유명합니다. 이 작품은 범죄와 인간 심리를 탐구하는 매력적인 이야기입니다.

4. **콜린 덕스 - "사라진 사람들"**: 현대 영국 추리소설의 대표작 중 하나로, 실종 사건을 중심으로 한 긴장감 넘치는 이야기입니다.

5. **루이스 캐럴 - "앨리스의 모험"**: 비록 전통적인 추리소설은 아니지만, 미스터리와 환상이 결합된 독특한 이야기로, 추리소설 팬에게도 흥미로울 수 있습니다.

이 중에서 마음에 드는 책이 있으면 좋겠네요! 더 많은 추천이 필요하시거나 특정한 스타일의 책을 원하시면 말씀해 

## 5. 출력 구조화를 위한 파서

### JSON Output Parser

In [ ]:
# JsonOutputParser
from langchain_core.output_parsers import JsonOutputParser
json_prompt = ChatPromptTemplate.from_template(
    """문장에서 '인물', '장소', '시간' 정보를 추출하여 JSON으로 출력해줘.
    문장: {sentence}
    JSON 출력:"""
)
json_chain = json_prompt | llm_openai | JsonOutputParser()
json_output = json_chain.invoke({"sentence": "내일 오후 3시에 서울역에서 김철수 씨를 만나기로 했어요."})
print(json_output)

{'인물': '김철수 씨', '장소': '서울역', '시간': '내일 오후 3시'}


### Pydantic Output Parser

In [ ]:
# PydanticOutputParser
from typing import List, Optional
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser

# 파서 클래스 정의
class MeetingDetails(BaseModel):
    location: Optional[str] = Field(description="회의가 열리는 장소")
    time: Optional[str] = Field(description="회의 시작 시간")
    attendees: List[str] = Field(description="회의 참석자들의 이름 목록", default=[])

# 파서 생성
pydantic_parser = PydanticOutputParser(pydantic_object=MeetingDetails)

# 프롬프트 템플릿 생성
pydantic_prompt = ChatPromptTemplate.from_template(
    """주어진 이메일 본문에서 회의 정보를 추출해줘.
    {format_instructions}
    이메일 본문: --- {email_body} ---
    """
    )

# 파서가 적용된 체인 생성
pydantic_chain = pydantic_prompt | llm_openai | pydantic_parser

In [111]:
sample_email = """
안녕하세요,
다음 주 프로젝트 리뷰 회의 일정을 공유드립니다.
- 일시: 2025년 8월 20일 (수) 오후 2시
- 장소: 본사 3층 대회의실
- 참석자: 김태균 팀장, 안가영 선임, 조예찬 책임
감사합니다.
"""
meeting_info = pydantic_chain.invoke({
    "email_body": sample_email,
    "format_instructions": pydantic_parser.get_format_instructions()
})
print(meeting_info)

location='본사 3층 대회의실' time='2025년 8월 20일 (수) 오후 2시' attendees=['김태균 팀장', '안가영 선임', '조예찬 책임']
